In [14]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.utils import to_categorical as tc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('C:\\Users\\ankur_vc1xnom\\Desktop\LP4\\LP-IV-datasets\\creditcard.csv\\creditcard.csv')

In [6]:
train.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [7]:
feat = train.drop('Class', axis = 1)
target = train['Class']

In [8]:
from sklearn.model_selection import train_test_split as tts

In [9]:
xtr, xts, ytr, yts = tts(feat,target, test_size = 0.2, random_state = 12)

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
mms = MinMaxScaler((0,1))

In [19]:
train_ind = ytr[ytr == 1].index
xtri = xtr.loc[train_ind]

In [20]:
xtrs = mms.fit_transform(xtri.copy())

In [21]:
xtrs.shape

(399, 30)

In [22]:
xtss = mms.transform(xts.copy())

In [32]:
class Aen(Model):
    def __init__(self, out, ldim = 8):
        super().__init__()
        self.encoder = Sequential([
            Dense(64, 'relu'),
            Dense(32, 'relu'),
            Dense(ldim, 'relu')
        ])
        self.decoder = Sequential([
            Dense(32, 'relu'),
            Dense(64, 'relu'),
            Dense(out, 'sigmoid')
        ])
    def call(self,inputs):
        en = self.encoder(inputs)
        de = self.decoder(en)
        return de

In [33]:
model = Aen(out = xtrs.shape[1])

In [53]:
model.compile(optimizer = 'adam', loss = 'mse', metrics = ['mse'])

In [54]:
history = model.fit(xtrs, xtrs, epochs=20, validation_data=(xtss, xtss))

Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 298ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.0087 - val_mse: 0.0087
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 270ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.0083 - val_mse: 0.0083
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 273ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.0081 - val_mse: 0.0081
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 276ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.0081 - val_mse: 0.0081
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 261ms/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.0081 - val_mse: 0.0081
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 277ms/step - loss: 0.0087 - mse: 0.0087 - val_loss: 0.0081 - val_mse: 0.0081
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 276ms/step - loss: 0.0089 - mse: 0.0089 - val_loss: 0.0081 - val_mse: 0.0081
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 261ms/step - loss: 0.0087 - mse: 0.0087 - val_loss: 0.0080 - val_mse: 0.0080
Epoch 9/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 276ms/step - lo

In [56]:
from tensorflow.keras.metrics import mse

In [57]:
def findth(model, xtr):
    recons = model.predict(xtr)
    er = mse(recons, xtr)
    th = np.mean(er.numpy()) + np.std(er.numpy())
    return th
thres = findth(model, xtr)

7121/7121 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step


In [58]:
thres

679158073.0160384

0

In [59]:
def get_pred(model, ther, xts):
    re = model.predict(xts)
    er = msle(re, xts)
    anomsk = pd.Series(er) > ther
    pred = anomsk.map(lambda x : 0.0 if x==True else 1.0)
    return pred

In [60]:
pred = get_pred(model, thres, xtss)

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step


In [61]:
from sklearn.metrics import accuracy_score

In [62]:
acc = accuracy_score(pred, yts)

In [63]:
acc

0.0016326673922966188

In [52]:
acc

0.0016326673922966188